IMPORT

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

TRAIN DATA:

In [147]:
# importing required modules 
from zipfile import ZipFile 


# opening the zip file in READ mode 
with ZipFile("/content/drive/My Drive/train.zip", 'r') as zip: 
	# printing all the contents of the zip file 
	zip.printdir() 

	# extracting all the files 
	print('Extracting all the files now...') 
	zip.extractall() 
	print('Done!') 

File Name                                             Modified             Size
train.csv                                      2020-03-19 00:50:22     88497206
Extracting all the files now...
Done!


In [0]:
data = pd.read_csv("/content/train.csv")

In [149]:
data.shape

(279, 60485)

Test data:

In [150]:
# importing required modules 
from zipfile import ZipFile 


# opening the zip file in READ mode 
with ZipFile("/content/drive/My Drive/test.zip", 'r') as zip: 
	# printing all the contents of the zip file 
	zip.printdir() 

	# extracting all the files 
	print('Extracting all the files now...') 
	zip.extractall() 
	print('Done!') 

File Name                                             Modified             Size
test.csv                                       2020-03-19 00:50:22     23186470
Extracting all the files now...
Done!


In [0]:
test_data = pd.read_csv("/content/test.csv")
test_data=test_data.drop(['id'], axis = 1)

PREPROCESSING:

In [0]:
labels=[]
labels=list(data["Label"])

In [153]:
data_new=data
data_new=data_new.drop(["id","Label"], axis = 1)
data_new.head()

ENSG00000242268.2  ENSG00000270112.3  ...  ENSG00000123685.7  ENSG00000181518.3
0           1.322600           0.000000  ...           1.708020                0.0
1           0.000000           0.006431  ...           0.182464                0.0
2           0.000000           0.000000  ...           0.386022                0.0
3           0.201191           0.000000  ...           0.666707                0.0
4           0.043040           0.000000  ...           0.513950                0.0

[5 rows x 60483 columns]

In [0]:
data_new=data_new.astype(np.float)

In [155]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_new,labels, test_size=0.1)
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",len(y_train))
print("y_test",len(y_test))
print("test_data",test_data.shape)

X_train (251, 60483)
X_test (28, 60483)
y_train 251
y_test 28
test_data (71, 60483)


NORMALIZATION:

In [0]:
# log2 normalise
X_train=np.log2(1+X_train)
X_test=np.log2(1+X_test)
test_data=np.log2(1+test_data)
# low variance normalisation

In [0]:
data_col=list(X_train.columns)
X_train_var=list(X_train.var())
for i in range(0,len(X_train_var)): 
   if X_train_var[i]<0.025:
     X_train=X_train.drop([data_col[i]],axis=1)
     X_test=X_test.drop([data_col[i]],axis=1)
     test_data=test_data.drop([data_col[i]],axis=1)


In [160]:
X_test.head()

ENSG00000242268.2  ...  ENSG00000123685.7
150           0.931163  ...           0.275476
51            0.346137  ...           0.541306
186           0.000000  ...           0.911998
114           0.000000  ...           0.172575
70            0.089278  ...           0.082131

[5 rows x 23674 columns]

In [342]:
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",len(y_train))
print("y_test",len(y_test))
print("test_data",test_data.shape)

X_train (251, 23674)
X_test (28, 23674)
y_train 251
y_test 28
test_data (71, 23674)


In [0]:
X_train_=X_train
X_test_=X_test
test_data_=test_data

In [344]:
test_data_.head()

ENSG00000242268.2  ENSG00000167578.15  ...  ENSG00000231119.2  ENSG00000123685.7
0           0.000000            0.522842  ...           0.164422           0.304786
1           0.193829            0.622900  ...           0.306519           0.322804
2           0.023347            0.579418  ...           0.180143           0.069272
3           0.000000            0.434230  ...           0.437276           0.430381
4           0.436196            0.351468  ...           0.128643           0.240638

[5 rows x 23674 columns]

In [0]:
# minmax normalise:
for col in X_train_.columns:
    if (X_train_[col].max() - X_train_[col].min())!=0:
      X_train_[col] = (X_train_[col] - X_train_[col].min())/ (X_train_[col].max() - X_train_[col].min())
    if (X_test_[col].max() - X_test_[col].min())!=0:
      X_test_[col] = (X_test_[col] - X_test_[col].min())/ (X_test_[col].max() - X_test_[col].min())
    if (test_data_[col].max() - test_data_[col].min())!=0:
      test_data_[col] = (test_data_[col] - test_data_[col].min())/ (test_data_[col].max() - test_data_[col].min())

FEATURE SELECTION:

In [0]:
colu_test_data=list(X_train_.columns)
S = SelectKBest(f_classif, k=100)
xkbest = S.fit_transform(X_train_,y_train)
features = S.get_support(indices=True)

In [0]:
xkbest.shape

(251, 100)

In [0]:
for i in colu_test_data:
  if colu_test_data.index(i) not in features:
      X_test_=X_test_.drop([i],axis=1)
      test_data_=test_data_.drop([i],axis=1)

In [349]:
test_data_.shape

(71, 100)

pca

In [0]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=70)
pca.fit(xkbest)
pca_train=pca.transform(xkbest)
pca_valid=pca.transform(X_test_)
pca_test=pca.transform(test_data_)

In [514]:
pca_valid

array([[-0.60504924, -0.32006457,  0.01993619, ..., -0.14490872,
         0.04636207,  0.00981235],
       [ 1.6716597 , -2.3152258 ,  0.24188404, ...,  0.05833874,
        -0.05305393,  0.15830937],
       [-0.32899242,  0.15580034, -0.27777762, ..., -0.20132829,
         0.13202972,  0.20102768],
       ...,
       [-0.84751724, -0.16300181,  0.17564335, ..., -0.15731578,
         0.06254486, -0.0543878 ],
       [-0.91237073, -0.08150171,  0.20942029, ..., -0.48243292,
        -0.22895499, -0.08576743],
       [-0.12412684,  0.86455215,  0.59926797, ..., -0.34202689,
         0.08186653,  0.0215821 ]])

In [0]:
new_pca=np.concatenate((pca_train,pca_valid))

In [518]:
new_pca.shape

(279, 70)

In [520]:
len(labels)

279

MODEL:

In [602]:
from sklearn.neural_network import MLPClassifier
X = new_pca
y = labels
clf = MLPClassifier(activation='tanh',solver='adam', alpha=1e-8,
                    hidden_layer_sizes=(6,2), random_state=0)

clf.fit(X, y)
y_pred=clf.predict(pca_test)
from sklearn.metrics import accuracy_score

y_true = [-1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1, -1,  1,  1,  1,  1,
        1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1, -1,  1, -1,
        1,  1,  1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  1,
        1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1, -1,
       -1, -1, -1]
accuracy_score(y_true, y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.6338028169014085

In [0]:
t_id= pd.read_csv("/content/test.csv")
id_=list(t_id['id'])
y_pred_=list(y_pred)
dic_={'id':id_,'Labels':y_pred_}
ans_=pd.DataFrame(dic_)
ans_.to_csv('new_output_level6.csv')

just for experiment: